In [1]:
import os
import numpy as np
import import_ipynb
from tqdm import tqdm as tqdm
import edsr

import matplotlib.image as mpimg
from PIL import Image
import cv2

from skimage.measure import compare_ssim as ssim
import video_to_images

importing Jupyter notebook from edsr.ipynb


/home/chandu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.layers import Conv2D

def compact(x_in):
    conv1 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(x_in)
    conv2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(conv1)
    last = Conv2D(1, strides = 2, kernel_size=(3,3), activation='sigmoid', padding = 'same')(conv2)
    return last

In [3]:
def compressThis(x_input, quality_factor):
    
    num = x_input.shape[0]
    ans = []
    for i in range(num):
        x_single = x_input[i,:,:,0]
        img = Image.fromarray(np.uint8(x_single*255))
        img.save(os.getcwd()+'/compress.JPEG',"JPEG", quality = quality_factor)
        out_single = mpimg.imread(os.getcwd()+'/compress.JPEG')
        ans.append(out_single)
    ans = np.array(ans)
    ans = np.reshape(ans, (ans.shape[0], ans.shape[1], ans.shape[2], 1))
    return ans

In [4]:
def load_images(dset_location):
    loc = dset_location
    images_location = []
    X = []

    print("Extracting image locations..")
    for i in tqdm(os.listdir(loc)):
        images_location.append(loc+'/'+i)

    print("Extracting images..")
    images_location = images_location[:images_count]
    for im_loc in tqdm(images_location):
        img = cv2.imread(im_loc)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        X.append(img)
    X = np.array(X)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))
    return X

In [5]:
def split_dataset(X, split_ratio):
    split = int(split_ratio * X.shape[0])
    x_train = X[:split,:,:,:]
    x_valid = X[split:,:,:,:]
    return x_train, x_valid

In [6]:
from keras.layers import Input
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam

def run():
    
    
    X = load_images(dset_location)
    x_train, x_valid = split_dataset(X, split_ratio)
    
    print("train images shape : " + str(x_train.shape))
    print("valid images shape : " + str(x_valid.shape))

    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    x_train = x_train / 255
    x_valid = x_valid / 255

    
    inp = Input(shape=(None, None, 1))
    model_edsr = edsr.edsr(x_in=inp, scale=2)
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.00000001, decay=0.0, amsgrad=False)
    model_edsr.compile(optimizer=adam, loss='mse')
    checkpointr = ModelCheckpoint(filepath='edsr.weights.arb.best.hdf5', save_best_only=True, verbose=1)
    model_edsr.summary()

    
    inp = Input(shape=(None, None, 1))
    model_comcnn = Model(inp, model_edsr(compact(inp)))
    model_comcnn.layers[4].trainable = False
    model_comcnn.compile(optimizer='adam', loss='mse')
    checkpointc = ModelCheckpoint(filepath='comcnn.weights.arb.best.hdf5', save_best_only=True, verbose=1)
    model_comcnn.summary()


    
    print("Entered into final training phase..")
    for i in range(iterations):
        
        print('Epoch : ' + str(i+1))
        
        upto_comcnn = Model(model_comcnn.input, model_comcnn.layers[3].output)
        xm = upto_comcnn.predict(x_train)
        xm_valid = upto_comcnn.predict(x_valid)
        mpimg.imsave('final-phase/'+str(i)+'-xm.png', xm[0,:,:,0], cmap='gray')
        
        xm_compress = compressThis(xm, QF)
        xm_valid_compress = compressThis(xm_valid, QF)
        xm_compress = xm_compress.astype('float32')
        xm_valid_compress = xm_valid_compress.astype('float32')
        xm_compress = xm_compress / 255
        xm_valid_compress = xm_valid_compress / 255
        mpimg.imsave('final-phase/'+str(i)+'-xm_compress.png', xm_compress[0,:,:,0], cmap='gray')
        
        print('Training edsr...')
        model_edsr.fit(x=xm_compress, y=x_train, validation_data=(xm_valid_compress, x_valid),epochs=num_of_epochs, shuffle=True, verbose=1, batch_size=8, callbacks=[checkpointr])
        print('Training comcnn...')
        model_comcnn.fit(x=x_train, y=x_train, validation_data=(x_valid, x_valid),epochs=num_of_epochs, shuffle=True, verbose=1, batch_size=8, callbacks=[checkpointc])
        
        sample_in = xm_compress[:1]
        sample_op = model_edsr.predict(sample_in)
        mpimg.imsave('edsr-check/'+str(i)+'-epoch-inp-epoch.png', sample_in[0,:,:,0], cmap='gray')
        mpimg.imsave('edsr-check/'+str(i)+'-epoch-out-epoch.png', sample_op[0,:,:,0], cmap='gray')
        
        sample_in = x_train[:1]
        sample_op = model_comcnn.predict(sample_in)
        mpimg.imsave('comcnn-check/'+str(i)+'-epoch-inp-epoch.png', sample_in[0,:,:,0], cmap='gray')
        mpimg.imsave('comcnn-check/'+str(i)+'-epoch-out-epoch.png', sample_op[0,:,:,0], cmap='gray')
        
        

In [7]:
dset_location ='/home/titanxpascal/Documents/sem6proj/img-compression/Subset16k/Subset16k'
images_count = 800
split_ratio = 0.8
QF = 10


# ----- final training phase ----------
iterations = 100
num_of_epochs = 5

lr_dim = 128
hr_dim = 256
